## Building your deep neural network: step by step

Welcome to your week 4 assignment (part 1 of 2)! You have previously trained a 2-layer Neural Network (with a single hidden layer). This week, you will build a deep neural network, with as many layers as you want!

  - In this notebook, you will implement all the functions required to build a deep neural network.
  - In the next assignment, you will use these functions to build a deep neural network for image classification.

#### Capacity:

1. Use non-linear units like ReLU to improve your model
2. Build a deeper neural network (with more than 1 hidden layer)
3. Implement an easy-to-use neural network class

### 1. Package

In [4]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v2 import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0)    # set default figure size of the plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 2. Outline of the assignment


To build your neural network, you will be implementing several "helper functions". These helper functions will be used in the next assignment to build a two-layer neural network and an L-layer neural network. Each small helper function you will implement will have detailed instructions that will walk you through the necessary steps. Here is an outline of this assignment, you will:

 - Initialize the parameters for two layer network and for an L layer neural network
 - Implemment the forward propagation module
  - Compute the LINEAR part of a layer's forward propagation step (resulting in $Z[L]$)
  - ACTIVATION function - ReLU and sigmoid
  - Combine the previous two steps into a new LINEAR and ACTIVATION forward function
  - Stack the LINEAR-ReLU forward function L-1 time and add a LINEAR sigmoid at the end.
 - Compute loss
 - Implement backward propagation module
  - Complete the LINEAR part of a layer's backward propagation step
  - Gradient of the ACTIVATE function (ReLU_backwards/sigmoid_backward)
  - Combine the previous two steps into a [LINEAR - ACTIVATE] backward function
  - Stack Linear-ReLU backward L-1 and add back to a new model
  - Finally update the parameters

### 3. Initialization

### 3.1 2-Layer Neural Network

- the model structure is Linear---ReLU---Linear---Sigmoid
- random initialization for the weight matrices. Use np.random.randn(shape) * 0.01
- use zero initialization for bias

In [13]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    parameters -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(1)
    
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y,1))
    
    
    parameters = {'W1': W1,
                  'b1': b1,
                  'W2': W2,
                  'b2': b2
    }
    
    return parameters

In [14]:
parameters = initialize_parameters(2,2,1)
print('W1 = ' + str(parameters['W1']))
print('b1 = ' + str(parameters['b1']))
print('W2 = ' + str(parameters['W2']))
print('b2 = ' + str(parameters['b2']))

W1 = [[ 0.01624345 -0.00611756]
 [-0.00528172 -0.01072969]]
b1 = [[0.]
 [0.]]
W2 = [[ 0.00865408 -0.02301539]]
b2 = [[0.]]


#### 3.2 L- layer neural network

   The initialization for a deeper L-layer neural network is more complicated because there are many more weight matrices and bias vectors. When completing the initialize_parameters_deep, you should make sure that your dimensions match between each layer. Recall that $n^{[l]}$ is the number of units in layer $l$. Thus for example if the size of our input $X$ is $(12288, 209)$ (with $m=209$ examples) then:

In [6]:
def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                 Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
    """
    np.random.seed(3)
    parameters = {}        # define a matrices
    L = len(layer_dims)    # number of layers in the network
    for i in range(1, L):
        ## Iterate from the first layer in the neural network
        parameters['W' + str(i)] = np.random.randn(layer_dims[i], layer_dims[i-1]) * 0.01
        parameters['b' + str(i)] = np.zeros((layer_dims[i], 1))
    return parameters

In [7]:
parameters = initialize_parameters_deep([5,4,3])  # (2 layer neural network)  # input 5, Layer 1 = 4, Layer 2 = 3
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


### 4. Forward propagation module

#### 4.1 - Linear Forward

- Linear
- Linear -- activate -- ReLU or Sigmoid
- Linear -- ReLU -- X -- Linear -- Sigmoid

$d(Sigmoid) = {a(1-a)}$

$d(ReLU) = {0 /  1}$